In [60]:
import numpy as np
import dense_basis as db
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, time, shutil, hickle
from tqdm import tqdm
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=1.8)
sns.set_style('white')
import hickle

import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import models

import sklearn
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.model_selection import train_test_split
from PIL import Image as image_PIL
from PIL import Image, ImageOps
import PIL
from PIL import ImageShow
from astropy.io import fits

from zoobot.pytorch.estimators import define_model as ZoobotModel

import shap



# Data Importing

In [61]:
work_dir = 'INSTERT DIRECTORY OF THE GITCLONED FOLDER HERE'

In [62]:
def resize_image(src_image, size=(256,256), bg_color="black"): 
    
    # resize the image so the longest dimension matches our target size
    src_image.thumbnail(size, Image.ANTIALIAS)
    
    # Create a new square background image
    new_image = Image.new("RGB", size, bg_color)
    
    # Paste the resized image into the center of the square background
    new_image.paste(src_image, (int((size[0] - src_image.size[0]) / 2), int((size[1] - src_image.size[1]) / 2)))
  
    # return the resized image
    return new_image

def load_image_data(idlist, size=(256,256), bg_color="black"):
    
    images = []
    for gal_id in tqdm(idlist):
        #img = plt.imread('images/'+gal_id+'.png')
        img = Image.open(work_dir+'images (DR17)/'+gal_id+'.png')
        img_sized = resize_image(img, size=size, bg_color=bg_color)
        images.append(img_sized)
    return images

class MaNGAdataset(Dataset):
    """images and labels for CNN"""
    
    def __init__(self, manga_cat, galmask, transform = None):
        
        gal_ids = manga_cat['manga_id']
        gal_ids = np.array(gal_ids,dtype=str)[galmask]
        gal_mass = manga_cat['log_mstar'][galmask]
        #gal_mass_scaled = (gal_mass - 9)/4
        gal_mass_scaled = gal_mass
        gal_sfr = manga_cat['log_sfr'][galmask]
        gal_z = manga_cat['redshift'][galmask]
        gal_n = manga_cat['sersic_n'][galmask]
        gal_t50_pipe3d = np.log10(manga_cat['t50_pipe3d'])[galmask]
        gal_t50_model = np.log10(manga_cat['t50_model'])[galmask]
        gal_d4000 = manga_cat['d4000'][galmask]
        gal_spirals = manga_cat['spirals'][galmask]
        gal_bars = manga_cat['bars'][galmask]
        gal_bulge = manga_cat['bulge'][galmask]
        gal_edge_on = manga_cat['edge_on'][galmask]
        gal_irregular_features = manga_cat['irregular_features'][galmask]

        # good_index = np.where(gal_sfr != -np.inf)[0]
        good_index = np.where(np.array(gal_d4000,dtype=str) != 'nan')[0] #cleaning out nan values of d4000 

        gal_ids = gal_ids[good_index]
        gal_mass = gal_mass[good_index]
        gal_sfr = gal_sfr[good_index]
        gal_z = gal_z[good_index]
        gal_n = gal_n[good_index]
        gal_t50_pipe3d = gal_t50_pipe3d[good_index]
        gal_t50_model = gal_t50_model[good_index]
        gal_d4000 = gal_d4000[good_index]
        gal_spirals = gal_spirals[good_index]
        gal_bulge = gal_bulge[good_index]
        gal_bars = gal_bars[good_index]
        gal_edge_on = gal_edge_on[good_index]
        gal_irregular_features = gal_irregular_features[good_index]


        ###########CLASSIC CLASSES#######################

        bars=np.linspace(np.min(gal_sfr), np.max(gal_sfr), num=11) #Number of classes set here (+1), default was 20 
        sfr_cat = np.zeros_like(gal_sfr)
        for i in range(len(bars)-1):
            sfr_cat[(gal_sfr > bars[i]) & (gal_sfr < bars[i+1])]=int(i)
        sfr_cat[gal_sfr > bars[len(bars)-1]] = int(len(bars)-1)

        ########################NEW MORE EVENLY DISTRIBUTED CLASSES###################
        
        # bars=[-6,-2.5,-2,-1.7,-1.5,-1.2,-0.9,-0.7,-0.4,-0.1,0.1,0.9]
        # sfr_cat = np.zeros_like(gal_sfr)
        # for i in range(len(bars)-1):
        #     sfr_cat[(gal_sfr > bars[i]) & (gal_sfr < bars[i+1])]=int(i)
        # sfr_cat[gal_sfr > bars[len(bars)-1]] = int(len(bars)-1)
      
        self.image = load_image_data(gal_ids)
        self.mangaid = gal_ids
        self.mstar = gal_mass
        self.mstar_class = gal_mass
        self.sfr = sfr_cat
        self.z = gal_z
        self.n = gal_n
        #self.c = ((gal_mass - 6)*3 - 1).astype(int)
        self.c = sfr_cat.astype(int)
        self.transform = transform
        self.sfr_value = gal_sfr
        self.t50_pipe3d = gal_t50_pipe3d
        self.t50_model = gal_t50_model
        self.d4000 = gal_d4000
        self.spirals=gal_spirals
        self.bars = gal_bars
        self.bulge = gal_bulge
        self.edge_on = gal_edge_on
        self.irregular_features = gal_irregular_features

        
    def __len__(self):
        return len(self.mstar)
    
    def __getitem__(self, idx):
    
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = {'image': self.image[idx],
                'mass': self.mstar[idx],
                'z':self.z[idx],
                'n':self.n[idx],
                'sfr_class':self.sfr[idx],
                'mangaid':self.mangaid[idx],
                'sfr':self.sfr_value[idx],
                't50_pipe3d':self.t50_pipe3d[idx],
                't50_model':self.t50_model[idx],
                'd4000':self.d4000[idx],
                'spirals':self.spirals[idx],
                'bars':self.bars[idx],
                'bulge':self.bulge[idx],
                'edge_on':self.edge_on[idx],
                'irregular_features':self.irregular_features[idx]}
        
        if self.transform:
            sample['image'] = self.transform(sample['image'])

            
            
        return sample

In [63]:
#Run this if you want both dataloaders to maintain the same order whenever they are iterated 

# train_loader = hickle.load(work_dir+'dataloaders/Train_Manga_DR17_log_t50_90_10_NO_SHUFFLE')
# val_loader = hickle.load(work_dir+'dataloaders/Test_Manga_DR17_log_t50_90_10')

#d4000 loaders
# train_loader = hickle.load(work_dir+'dataloaders/Train_Manga_DR17_d4000_NO_SHUFFLE')
# val_loader = hickle.load(work_dir+'dataloaders/Test_Manga_DR17_d4000')

# #d4000 loaders with no transformations applied to the images 
train_loader = hickle.load(work_dir+'dataloaders/Train_Manga_DR17_Morph_for_SHAP_MAPS')
val_loader = hickle.load(work_dir+'dataloaders/Test_Manga_DR17_Morph_for_SHAP_MAPS')

In [64]:
batch_size = 32

train_ids = []
mass = []
sfr = []
t50_pipe3d = []
t50_model = []
images = []
z = []
d4000 = []
spirals = []
bars = []
bulge = []
edge_on = []
irregular_features = []

data_sample = train_loader

for batch_idx, temp in enumerate(data_sample):
    train_ids.append(temp['mangaid'])
    mass.append(temp['mass'])
    sfr.append(temp['sfr'])
    t50_pipe3d.append(temp['t50_pipe3d'])
    t50_model.append(temp['t50_model'])
    images.append(temp['image'])
    z.append(temp['z'])
    d4000.append(temp['d4000'])
    spirals.append(temp['spirals'])
    bars.append(temp['bars'])
    bulge.append(temp['bulge'])
    edge_on.append(temp['edge_on'])
    irregular_features.append(temp['irregular_features'])


incomplete_batch_id = len(train_ids) - 1
remainder = len(train_ids[incomplete_batch_id])

total_values = (len(train_ids) * batch_size) - (batch_size - remainder)

train_ids_list = []
train_mass_list = []
train_sfr_list = []
train_t50_pipe3d_list = []
train_t50_model_list = []
train_images_list = []
train_z_list = []
train_d4000_list = []
train_spirals_list = []
train_bars_list = []
train_bulge_list = []
train_edge_on_list = []
train_irregular_features_list = []

k = 0
while k < total_values:
    for i in range(len(train_ids)):
        if i < incomplete_batch_id:
            for j in range(batch_size):
                train_ids_list.append(train_ids[i][j])
                train_mass_list.append(mass[i][j])
                train_sfr_list.append(sfr[i][j])
                train_t50_pipe3d_list.append(t50_pipe3d[i][j])
                train_t50_model_list.append(t50_model[i][j])
                train_images_list.append(images[i][j])
                train_z_list.append(z[i][j])
                train_d4000_list.append(d4000[i][j])
                train_spirals_list.append(spirals[i][j])
                train_bars_list.append(bars[i][j])
                train_bulge_list.append(bulge[i][j])
                train_edge_on_list.append(edge_on[i][j])
                train_irregular_features_list.append(irregular_features[i][j])
                k += 1
        else:
            for j in range(remainder):
                train_ids_list.append(train_ids[i][j])
                train_mass_list.append(mass[i][j])
                train_sfr_list.append(sfr[i][j])
                train_t50_pipe3d_list.append(t50_pipe3d[i][j])
                train_t50_model_list.append(t50_model[i][j])
                train_images_list.append(images[i][j])
                train_z_list.append(z[i][j])
                train_d4000_list.append(d4000[i][j])
                train_spirals_list.append(spirals[i][j])
                train_bars_list.append(bars[i][j])
                train_bulge_list.append(bulge[i][j])
                train_edge_on_list.append(edge_on[i][j])
                train_irregular_features_list.append(irregular_features[i][j])
                k += 1


In [65]:
batch_size = 32

val_ids = []
mass = []
sfr = []
t50_pipe3d = []
t50_model = []
images = []
z = []
d4000 = []
spirals = []
bars = []
bulge = []
edge_on = []
irregular_features = []

data_sample = val_loader

for batch_idx, temp in enumerate(data_sample):
    val_ids.append(temp['mangaid'])
    mass.append(temp['mass'])
    sfr.append(temp['sfr'])
    t50_pipe3d.append(temp['t50_pipe3d'])
    t50_model.append(temp['t50_model'])
    images.append(temp['image'])
    z.append(temp['z'])
    d4000.append(temp['d4000'])
    spirals.append(temp['spirals'])
    bars.append(temp['bars'])
    bulge.append(temp['bulge'])
    edge_on.append(temp['edge_on'])
    irregular_features.append(temp['irregular_features'])


incomplete_batch_id = len(val_ids) - 1
remainder = len(val_ids[incomplete_batch_id])

total_values = (len(val_ids) * batch_size) - (batch_size - remainder)

test_ids_list = []
test_mass_list = []
test_sfr_list = []
test_t50_pipe3d_list = []
test_t50_model_list = []
test_images_list = []
test_z_list = []
test_d4000_list = []
test_spirals_list = []
test_bars_list = []
test_bulge_list = []
test_edge_on_list = []
test_irregular_features_list = []

k = 0
while k < total_values:
    for i in range(len(val_ids)):
        if i < incomplete_batch_id:
            for j in range(batch_size):
                test_ids_list.append(val_ids[i][j])
                test_mass_list.append(mass[i][j])
                test_sfr_list.append(sfr[i][j])
                test_t50_pipe3d_list.append(t50_pipe3d[i][j])
                test_t50_model_list.append(t50_model[i][j])
                test_images_list.append(images[i][j])
                test_z_list.append(z[i][j])
                test_d4000_list.append(d4000[i][j])
                test_spirals_list.append(spirals[i][j])
                test_bars_list.append(bars[i][j])
                test_bulge_list.append(bulge[i][j])
                test_edge_on_list.append(edge_on[i][j])
                test_irregular_features_list.append(irregular_features[i][j])
                k += 1
        else:
            for j in range(remainder):
                test_ids_list.append(val_ids[i][j])
                test_mass_list.append(mass[i][j])
                test_sfr_list.append(sfr[i][j])
                test_t50_pipe3d_list.append(t50_pipe3d[i][j])
                test_t50_model_list.append(t50_model[i][j])
                test_images_list.append(images[i][j])
                test_z_list.append(z[i][j])
                test_d4000_list.append(d4000[i][j])
                test_spirals_list.append(spirals[i][j])
                test_bars_list.append(bars[i][j])
                test_bulge_list.append(bulge[i][j])
                test_edge_on_list.append(edge_on[i][j])
                test_irregular_features_list.append(irregular_features[i][j])
                k += 1


In [66]:
len(train_ids_list)+len(test_ids_list)

9414

In [9]:
# work_dir = '/home/juanpabloalfonzo/Documents/Manga CNNs/'
model_used = 'ResNet50_log_t50_chain_90_10'

device = 'cuda'

model_used = 'ResNet50_log_t50_chain_90_10'
model_mass = models.resnet50(weights=None)
model_mass.fc = nn.Linear(2048, 1)
model_mass = nn.DataParallel(model_mass,device_ids=(0,1))
model_mass.load_state_dict(torch.load(work_dir+'models/Mass_'+model_used+'.pytorch'),strict=True)#strict is set to false since it was trained on multiple GPUs it causes an error when loaded on the model that is not on multiple GPUs yet
                                                                                                # DO NOT DO THIS!! MESSES WITH THE MODEL PREDECTIONS HEAVILY
model_mass.eval()

model_sfr = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')
model_sfr.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3,bias=False)
model_sfr.fc = nn.Linear(2048, 1)
model_sfr = nn.DataParallel(model_sfr,device_ids=(0,1))
model_sfr.load_state_dict(torch.load(work_dir+'models/SFR_'+model_used+'.pytorch'),strict=True)
model_sfr.eval()

model_used = 'd4000_chain'
model_t50 = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')
model_t50.conv1 = nn.Conv2d(5, 64, kernel_size=7, stride=2, padding=3,bias=False)
model_t50.fc = nn.Linear(2048, 1)
model_t50 = nn.DataParallel(model_t50,device_ids=(0,1))
model_t50.load_state_dict(torch.load(work_dir+'models/d4000_'+model_used+'.pytorch'),strict=True)
model_t50.eval()



model_mass, model_sfr, model_t50 = model_mass.to(device), model_sfr.to(device), model_t50.to(device)


model_d4000 = models.resnet50(weights=None)
model_d4000.fc = nn.Linear(2048, 1)
model_d4000.load_state_dict(torch.load(work_dir+'models/d4000.pytorch'),strict=True)
model_d4000.to('cuda')
model_d4000.eval()

model_sfr_solo = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')
model_sfr_solo.fc = nn.Linear(2048,1)
model_sfr_solo.load_state_dict(torch.load(work_dir+'models/sfr_solo.pytorch'),strict=True)

<All keys matched successfully>

# D4000 to t50 Pipeline

In [13]:
def calc_d4000(lam, spec):
    # this is probably not quite correct, 
    # look up the defn in a paper and modify accordingly
    # also make sure spec is in the right units!!!
    
    lam_mask1 = (lam > 3850) & (lam < 3950)
    lam_mask2 = (lam > 4100) & (lam < 4200)
    dn4000 = np.nanmean(spec[lam_mask2]) / np.nanmean(spec[lam_mask1])
    return dn4000

In [69]:
def make_dn4000_for_synthetic_obs(mstar, sfr, t50, Av, Z, zval):
    sfh_tuple = np.array([mstar, sfr, 1.0, t50])
    specdetails = [sfh_tuple, Av, Z, zval]
    lam, spec = db.makespec(specdetails, priors, db.mocksp, db.cosmo, return_spec = True, peraa=True)
    dn4000 = calc_d4000(lam, spec)
    return dn4000

# Making Mock Observations

Distrubution of MaNGA Dataset

In [74]:
#Getting additional data to create mock MaNGA Spectrum from PIPE3D VAC
data = fits.open('/home/juanpabloalfonzo/Documents/Manga CNNs/SDSS17Pipe3D_v3_1_1.fits')
table = data[1]

Av=[]
Z=[]
manga_sfr =[]
manga_mass = []
for i in range(len(table.data)):
    Av.append(table.data[i][173])
    Z.append(table.data[i][28])
    manga_mass.append(table.data[i][12])
    manga_sfr.append(table.data[i][7])




In [ ]:
sample_size= 10000

t50_vals = np.random.uniform(low = 10**(np.min([np.min(train_t50_list),np.min(test_t50_list)]))/13.6, high =10**(np.max([np.max(train_t50_list),np.max(test_t50_list)]))/13.6,size=(sample_size,))
# t50_vals = np.random.uniform(low = 0.05, high = 0.81, size=(1000,))
# mstar_vals = np.random.uniform(low = np.min([np.min(train_mass_list),np.min(test_mass_list)]), high =np.max([np.max(train_mass_list),np.max(test_mass_list)]), size=(sample_size,))
mstar_vals = np.random.uniform(8,12.5,size=(sample_size,))
# sfr_vals = np.random.uniform(low = np.min([np.min(train_sfr_list),np.min(test_sfr_list)]), high =np.max([np.max(train_sfr_list),np.max(test_sfr_list)]), size=(sample_size,))
sfr_vals = np.random.uniform(-4,2.5,size=(sample_size,))
Av_vals = np.random.uniform(low = np.nanmin(Av), high =np.nanmax(Av), size=(sample_size,))
Z_vals = np.random.uniform(low = np.nanmin(Z), high =np.nanmax(Z), size=(sample_size,))
redshift = np.random.uniform(low = np.min([np.min(train_z_list),np.min(test_z_list)]), high =np.max([np.max(train_z_list),np.max(test_z_list)]), size=(sample_size,))

priors = db.Priors()


dn4000_vals = np.zeros_like(t50_vals)
for i in db.tqdm(range(len(t50_vals))):
    dn4000_vals[i] = make_dn4000_for_synthetic_obs(mstar_vals[i], sfr_vals[i], t50_vals[i], Av[i], Z[i], redshift[i])
    
plt.plot(t50_vals, dn4000_vals,'.')
plt.show()



# T50 Network

In [45]:
# Define the one-layer linear neural network class
class LinearNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LinearNet, self).__init__()
        self.linear_in = nn.Linear(input_size, hidden_size)
        self.linear_out = nn.Linear(hidden_size, output_size)
        self.linear = nn.Linear(hidden_size,hidden_size)
        self.activation = nn.ReLU()
    
    def forward(self, x):
        # print(x.size())
        out = self.linear_in(x)
        # print(out.size())
        out = self.activation(out)
        out = self.linear(out)
        out = self.activation(out)
        out = self.linear(out)
        out = self.activation(out)
        out = self.linear(out)
        out = self.activation(out)
        # out = self.linear(out)
        # out = self.activation(out)
        out = self.linear_out(out)
        return out

# Example usage
input_size = 2
hidden_size = 100
output_size = 1


# Create an instance of the LinearNet
d4000_to_t50 = LinearNet(input_size, hidden_size, output_size).to('cuda')


In [47]:
# Define loss function and optimizer
device = 'cuda'
criterion = nn.MSELoss()
optimizer = optim.Adam(d4000_to_t50.parameters(), lr=0.001)

t50_tensor = torch.tensor(t50_vals).unsqueeze(1).to('cuda')
dn4000_tensor = torch.tensor(dn4000_vals).unsqueeze(1).to('cuda')

input_data = torch.tensor(dn4000_vals, dtype=torch.float32).unsqueeze(1)
input_data1 = torch.tensor(mstar_vals, dtype=torch.float32).unsqueeze(1)
input_data2 = torch.tensor(sfr_vals, dtype=torch.float32).unsqueeze(1)
target_data = torch.tensor(t50_vals, dtype=torch.float32).unsqueeze(1) 

dataset = TensorDataset(input_data, input_data1, input_data2, target_data)

train_ratio = 0.8

# Calculate the number of samples for train and test
num_samples = len(dataset)
num_train = int(train_ratio * num_samples)
num_test = num_samples - num_train

# Split the dataset into train and test
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [num_train, num_test])

batch_size=32
# Create train and test loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Training loop
epochs = 400
for epoch in tqdm(range(epochs)):
    # Set model to training mode
    d4000_to_t50.train()

    # Iterate over the train loader
    for batch in train_loader:
        inputs0, inputs1, inputs2, targets = batch
        inputs = torch.stack((inputs0,inputs2-inputs1),dim=1).reshape(batch_size,input_size)
        inputs = inputs.reshape(inputs.shape[0],inputs.shape[1])
        # .swapaxes(1,2)
        # inputs = inputs0
        # print(inputs.size())
        inputs = inputs.to(device)
        targets = targets
        # print(targets.size())
        targets = targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = d4000_to_t50(inputs)
        
        # Calculate loss
        loss = criterion(outputs, targets)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
    
    # Print progress
    if (epoch+1) % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss.item()))

    # Testing loop
    with torch.no_grad():
        # Set model to evaluation mode
        d4000_to_t50.eval()

        test_loss = 0.0
        total_samples = 0

        for batch in test_loader:
            inputs0, inputs1, inputs2, targets = batch
            inputs = torch.stack((inputs0,inputs2-inputs1),dim=1)
            inputs = inputs.reshape(inputs.shape[0],inputs.shape[1])
            # inputs = inputs0
            inputs = inputs.to(device)
            targets = targets
            targets = targets.to(device)


            # Forward pass on test data
            test_outputs = d4000_to_t50(inputs)

            # Calculate test loss
            batch_loss = criterion(test_outputs, targets)
            test_loss += batch_loss.item() * len(inputs)
            total_samples += len(inputs)

        # Calculate and print the average test loss
        average_test_loss = test_loss / total_samples
        # print("Average Test Loss:", average_test_loss)


     
       

 25%|██▌       | 100/400 [01:31<04:42,  1.06it/s]

Epoch [100/400], Loss: 0.0325


 50%|█████     | 200/400 [03:03<03:06,  1.07it/s]

Epoch [200/400], Loss: 0.0228


 75%|███████▌  | 300/400 [04:35<01:31,  1.10it/s]

Epoch [300/400], Loss: 0.0369


100%|██████████| 400/400 [05:59<00:00,  1.11it/s]

Epoch [400/400], Loss: 0.0158
